In [1]:
from opentrons import simulate, execute, protocol_api
import pandas as pd
import numpy as np
import json
import os
import glob

C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults


Loading json containers...
Json container file load complete, listing database
Found 0 containers to add. Starting migration...
Database migration complete!


In [2]:
# protocol_api.labware.get_all_labware_definitions()

In [3]:
def custom_labware_dict(labware_dir_path):
    """Given the path of a folder of custom labware .json files will create dict
    of key = name and value = labware definition to be loaded using protocol.load_labware_from_definition 
    versus typical protocol.load_labware"""
    os.chdir(labware_dir_path)
    labware_dict = {}
    for file in glob.glob("*.json"):
        with open(file) as labware_file:
            labware_name = os.path.splitext(file)[0] # removes the .json extnesion
            labware_def = json.load(labware_file)
            labware_dict[labware_name] = labware_def
    return labware_dict 

In [6]:
def run(protocol, labware_dict):
    protocol.home()

    metadata = {
    'protocolName': 'My Protocol',
    'author': 'Name <email@address.com>',
    'description': 'Simple protocol to get started using OT2',
    'apiLevel': '2.5'
    }

    labware_name = '20mlscintillation_12_wellplate_18000ul'
    
    plate = protocol.load_labware_from_definition(labware_dict[labware_name], 1)
    print(plate)
    # plate = protocol.load_labware('corning_96_wellplate_360ul_flat', '11')
    tiprack = protocol.load_labware('opentrons_96_tiprack_300ul', '10')

    hold = [j for i in plate.rows() for j in i]
    left_pipette = protocol.load_instrument('p300_single', 'left', tip_racks=[tiprack])

    left_pipette.pick_up_tip()
    left_pipette.aspirate(100, hold[0])
    left_pipette.dispense(100, hold[1])
    left_pipette.drop_tip()



    for line in protocol.commands():
        print(line)

In [7]:
def custom_or_native_labware(protocol, labware_name, labware_slot, custom_labware_dict):
    if labware_name in custom_labware_dict:
        loaded_labware = protocol.load_labware_from_definition(custom_labware_dict[labware_name], labware_slot)
    else: 
        loaded_labware = protocol.load_labware(labware_name, labware_slot)     
    return loaded_labware

In [8]:
labware_dir_path = r"C:\Users\Edwin\Desktop\OT2Protocols\ot2protocol\Ouzo_OT2_Sampling\Custom Labware"
labware_dict = custom_labware_dict(labware_dir_path)
protocol = simulate.get_protocol_api('2.0')
test_name_custom = '20mlscintillation_12_wellplate_18000ul'
test_name_native = "corning_96_wellplate_360ul_flat"
test_slot_custom = "1"
test_slot_native = "2"
custom = custom_or_native_labware(protocol, test_name_custom, test_slot_custom, labware_dict)
native = custom_or_native_labware(protocol, test_name_native, test_slot_native, labware_dict)

C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults


In [18]:
import OT2_Ouzo_Commands_API2c as ot2_commands   

ModuleNotFoundError: No module named 'OT2_Ouzo_Commands_API2c'

In [23]:
labware_dir_path = r"C:\Users\Edwin\Desktop\OT2Protocols\ot2protocol\Ouzo_OT2_Sampling\Custom Labware"
labware_dict = custom_labware_dict(labware_dir_path)
labware_dict

{'20mlscintillation_12_wellplate_18000ul': {'ordering': [['A1', 'B1', 'C1'],
   ['A2', 'B2', 'C2'],
   ['A3', 'B3', 'C3'],
   ['A4', 'B4', 'C4']],
  'brand': {'brand': '20mLscintillation', 'brandId': []},
  'metadata': {'displayName': '20mLscintillation 12 Well Plate 18000 ÂµL',
   'displayCategory': 'wellPlate',
   'displayVolumeUnits': 'ÂµL',
   'tags': []},
  'dimensions': {'xDimension': 127.7, 'yDimension': 85.8, 'zDimension': 60.5},
  'wells': {'A1': {'depth': 53,
    'totalLiquidVolume': 18000,
    'shape': 'circular',
    'diameter': 27.85,
    'x': 21.81,
    'y': 70.69,
    'z': 7.5},
   'B1': {'depth': 53,
    'totalLiquidVolume': 18000,
    'shape': 'circular',
    'diameter': 27.85,
    'x': 21.81,
    'y': 42.86,
    'z': 7.5},
   'C1': {'depth': 53,
    'totalLiquidVolume': 18000,
    'shape': 'circular',
    'diameter': 27.85,
    'x': 21.81,
    'y': 15.03,
    'z': 7.5},
   'A2': {'depth': 53,
    'totalLiquidVolume': 18000,
    'shape': 'circular',
    'diameter': 27.

In [63]:
protocol = simulate.get_protocol_api('2.0', extra_labware=labware_dict)
plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)
dicty = plate.__dict__
type(dicty['_well_definition']['A1']['totalLiquidVolume'])

C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults


int

In [ ]:
def determines_stock_setup(protocol, experiment_dict, sample_volumes_df):
    stock_plate = custom_or_native_labware(protocol, experiment_dict['OT2 Stock Labware'], experiment_dict['OT2 Stock Labware Slot'], custom_labware_dict)
    stock_plate_rows = [well for row in stock_plate.rows() for well in row]
    stock_plate_well_volume = stock_plate.__dict__['_well_definition']['A1']['totalLiquidVolume']
    print(stock_plate_well_volume)

In [40]:
protocol = simulate.get_protocol_api('2.0', extra_labware=labware_dict)
def run(protocol):
    metadata = {'apiLevel': '2.8'}
    plate = protocol.load_labware('20mlscintillation_12_wellplate_18000ul', 1)
    print(len(plate.wells()))
    print(plate.__dict__)
    
    tiprack = protocol.load_labware('opentrons_96_tiprack_300ul', 2)
    tiprack_multi = protocol.load_labware('opentrons_96_tiprack_300ul', 3)
    pipette = protocol.load_instrument('p300_single', mount='left', tip_racks=[tiprack])
    pipette_multi = protocol.load_instrument('p300_multi', mount='right', tip_racks=[tiprack_multi])
    
    pipette.transfer(100, plate.wells_by_name()['A1'], plate.wells_by_name()['B1'])
    for line in protocol.commands():
        print(line)

C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults


In [41]:
run(protocol)

12
{'_api_version': APIVersion(major=2, minor=2), '_name': '20mlscintillation_12_wellplate_18000ul', '_display_name': '20mLscintillation 12 Well Plate 18000 ÂµL on 1', '_calibrated_offset': Point(x=0.0, y=0.0, z=0), '_wells': [A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1, B1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1, C1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1, A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1, B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1, C2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1, A3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1, B3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1, C3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1, A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1, B4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1, C4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1], '_well_definition': {'A1': {'depth': 53, 'totalLiquidVolume': 18000, 'shape': 'circular', 'diameter': 27.85, 'x': 21.81, 'y'